In [17]:
import torch
from torch import nn
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 🧠 **Many Men vs Iron Man** 🧠

### Welcome to an Lesson 03! 🚀

In this notebook, we will pit an ensemble of 3 neural networks 👫👫👫 against 1 big neural network 💪 to see which performs better at identifying the state (REM, wakeful, deep) of a 10-second long segment of data.

Our data comes from **rats** 🐀 and includes 10-second long segments that are classified into one of three states: REM, wakeful, or deep sleep. By comparing these two approaches, we aim to determine whether the combined wisdom of many smaller models or the sheer power of a single large model will triumph.

Will the collective intelligence of many prevail, or will the might of a lone giant come out on top? Let's dive in and find out! 🔍⚡

In [20]:
DATA_FOLDER = './../data/rat_eeg'


files = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.pt')]

all_data = []
for file in files:
    path = os.path.join(DATA_FOLDER, file)
    data = torch.load(path)
    all_data.append((data[0], data[1]))

# split  into train and test sets
train_data, test_data = train_test_split(all_data, test_size=2/6, random_state=42)

# concat to creat full set
X_train = torch.cat([data[0] for data in train_data])
y_train = torch.cat([data[1] for data in train_data])

X_test = torch.cat([data[0] for data in test_data])
y_test = torch.cat([data[1] for data in test_data])

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")


Training data shape: torch.Size([34560, 1000])
Training labels shape: torch.Size([34560, 3])
Test data shape: torch.Size([17280, 1000])
Test labels shape: torch.Size([17280, 3])


### 🎵 Side Note 🎵

Can we code this whole thing up while listening to nothing but "Many Men"? 🤔🎶 I don't know, but I will keep you updated at eh end of the note book!
